In [1]:
# standard python imports
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import copy

# pytorch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [2]:
data = np.load('exp_io_data\CoupledPendula_mean_in7_out7_Tmax1.0_data.npz')
for key, val in data.items():
    print(key)
    exec(key +'=val')

xlist
exp_out_list


In [3]:
x=xlist
y=exp_out_list
print(x.shape)
print(y.shape)

(2000, 14)
(2, 30, 2000, 7, 2)


In [5]:
x_in=x[:,0:7]
x_para=x[:,7:14]
#print(x_in)
#print(x_para)
v_int=np.zeros((2000,7))
x_train=x_in[0:1000,:]
x_train2=x[0:1000,:]
print(x_train.shape)
#print(x)

(1000, 7)


In [6]:

y_temp2=y[0,0:29,:,:,0]
y_temp1=y[0,1:30,:,:,0]
print(y_temp2.shape)
print(y_temp1.shape)
y_v=(y_temp2-y_temp1)*30
y_0=np.zeros((1,2000,7))
y_v=np.concatenate((y_0,y_v),axis=0)
print(y_v.shape)

(29, 2000, 7)
(29, 2000, 7)
(30, 2000, 7)


In [7]:
#y1=y[0:1,29:30,:,:,0:1]
print(y.shape)
y_x=y[0,:,:,:,0]
y_new=np.concatenate((y_x,y_v),axis=2)
y_new=torch.from_numpy(y_new)
y_new=torch.transpose(y_new,0,1)
print(y_new.shape)


(2, 30, 2000, 7, 2)
torch.Size([2000, 30, 14])


In [8]:
F=3 #Fth frame

y_train=y_new[0:1000,F:F+1,:]
print(y_train.shape)

torch.Size([1000, 1, 14])


In [30]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(14, 14 ,bias=True)
        self.layer2 = nn.Linear(14, 14,bias=True)
        self.layer3 = nn.Linear(14, 14,bias=True)
        #self.layer4 = nn.Linear(14, 14,bias=True)
        #self.layer5 = nn.Linear(14, 14,bias=True)
        self.layer4 = nn.Linear(14, 14,bias=True)
    def forward(self, x,activation="RELU"):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = x+torch.sigmoid(self.layer1(x))
        x = x+torch.sigmoid(self.layer2(x))
        x = x+torch.sigmoid(self.layer3(x))
        #x = x+torch.sigmoid(self.layer4(x))
        #x = x+torch.sigmoid(self.layer5(x))
        x = self.layer4(x)
        #x = F.relu(self.layer4(x))
        #x = F.relu(self.layer5(x))
        #x = torch.sigmoid(self.layer6(x))
        return x.float()


net = Net()
print(net)

Net(
  (layer1): Linear(in_features=14, out_features=14, bias=True)
  (layer2): Linear(in_features=14, out_features=14, bias=True)
  (layer3): Linear(in_features=14, out_features=14, bias=True)
  (layer4): Linear(in_features=14, out_features=14, bias=True)
)


In [26]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Linear(14, 14 ,bias=True)
        self.layer2 = nn.Linear(14, 14,bias=True)
        self.layer3 = nn.Linear(14, 14,bias=True)
        #self.layer4 = nn.Linear(14, 14,bias=True)
        #self.layer5 = nn.Linear(14, 14,bias=True)
        self.layer4 = nn.Linear(14, 7,bias=True)
    def forward(self, x,activation="RELU"):
        
        if activation=="RELU":
            x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
            x = x+0.1*(self.layer1(x))
            x = x+0.1*(self.layer2(x))
            x = x+0.1*(self.layer3(x))
            #x = x+torch.sigmoid(self.layer4(x))
            #x = x+torch.sigmoid(self.layer5(x))
            x = self.layer4(x)
            #x = F.relu(self.layer4(x))
            #x = F.relu(self.layer5(x))
            #x = torch.sigmoid(self.layer6(x))
        if activation=="SIG":
            x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
            x = x+torch.sigmoid(self.layer1(x))
            x = x+torch.sigmoid(self.layer2(x))
            x = x+torch.sigmoid(self.layer3(x))
            #x = x+torch.sigmoid(self.layer4(x))
            #x = x+torch.sigmoid(self.layer5(x))
            x = self.layer4(x)
            #x = F.relu(self.layer4(x))
            #x = F.relu(self.layer5(x))
            #x = torch.sigmoid(self.layer6(x))            
        return x.float()


net = Net()
print(net)

Net(
  (layer1): Linear(in_features=14, out_features=14, bias=True)
  (layer2): Linear(in_features=14, out_features=14, bias=True)
  (layer3): Linear(in_features=14, out_features=14, bias=True)
  (layer4): Linear(in_features=14, out_features=7, bias=True)
)


In [31]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

8
torch.Size([14, 14])


In [32]:
x_data=torch.tensor(x_train2)
input = x_data
out = net(input,"SIG")
print(out)
print(out.shape)

tensor([[-2.9287,  0.8485,  2.9031,  ...,  1.4912,  0.5082,  1.9616],
        [-2.9467,  0.5268,  2.8854,  ...,  1.4895,  0.4990,  1.9502],
        [-2.9327,  0.9733,  2.9170,  ...,  1.2457,  0.2563,  2.2040],
        ...,
        [-2.9613,  0.4369,  2.8786,  ...,  1.4425,  0.3197,  1.8224],
        [-3.1352,  0.3811,  2.9775,  ...,  1.4827,  0.1828,  1.9060],
        [-3.0442,  0.6493,  2.7635,  ...,  1.5666,  0.3755,  2.1952]],
       grad_fn=<AddmmBackward>)
torch.Size([1000, 14])


In [33]:
net.zero_grad()
out.backward(torch.randn(1000,14))

In [34]:
target=y_train
target_new=target.view(1000,14)
print(target_new.shape)
criterion=nn.MSELoss()
loss=criterion(out,target_new)
print(loss)

torch.Size([1000, 14])
tensor(27.5810, dtype=torch.float64, grad_fn=<MseLossBackward>)


In [37]:
print((target_new))
output = net(input)
print((output))
print((target_new.shape))
print((output.shape))

optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target_new)
print(loss)

tensor([[ -0.6331,  -1.0633,   0.6323,  ..., -11.2331,  15.7281,  -4.7278],
        [  0.5520,   0.3416,   0.4333,  ...,  -0.1292,  -6.7235,   5.0746],
        [  0.1080,   0.5514,  -0.8419,  ...,   3.8873,   1.0384,  -7.9504],
        ...,
        [  0.4511,  -0.8309,  -0.5465,  ...,   7.9163,  -9.3738,  -3.0244],
        [  1.0231,  -0.1878,  -0.0298,  ...,  11.8963,  -8.0969,  -0.2513],
        [ -0.0641,   0.4567,   0.9569,  ...,   4.4670,  -2.0576,  -4.3529]],
       dtype=torch.float64)
tensor([[-2.9287,  0.8485,  2.9031,  ...,  1.4912,  0.5082,  1.9616],
        [-2.9467,  0.5268,  2.8854,  ...,  1.4895,  0.4990,  1.9502],
        [-2.9327,  0.9733,  2.9170,  ...,  1.2457,  0.2563,  2.2040],
        ...,
        [-2.9613,  0.4369,  2.8786,  ...,  1.4425,  0.3197,  1.8224],
        [-3.1352,  0.3811,  2.9775,  ...,  1.4827,  0.1828,  1.9060],
        [-3.0442,  0.6493,  2.7635,  ...,  1.5666,  0.3755,  2.1952]],
       grad_fn=<AddmmBackward>)
torch.Size([1000, 14])
torch.Size([1

In [36]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.01, betas=(0.9, 0.999), eps=1e-09, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)


RuntimeError: Found dtype Double but expected Float

In [ ]:
x_test=x[1000:2000,:]
print(x_test.shape)
y1=y[0:1,:,1000:2000,:,0:1]
#(Nrepeat, Ntimesteps, Nx, output_dim, 2)  <-- 2 = (final position, final velocity)
y1_np=torch.from_numpy(y1)
y1=torch.transpose(y1_np,1,2)

In [ ]:
y4=y1.squeeze(0).squeeze(3)
print(y4.shape)

y_test=y4[:,F:F+1,:]
print(y_test.shape)

In [ ]:
x_test=torch.tensor(x_test)
test_out=net(x_test)
#LOSS = criterion(test_out, y_test)
print(test_out)
print(y_test)
y_test_new=torch.transpose(y_test,1,2).squeeze(2)
test_loss=criterion(y_test_new,test_out)
print(test_loss)

In [ ]:
plt.plot(y_test_new.flatten(), 5*test_out.detach().numpy().flatten(), '.')
plt.title('Training')
plt.xlabel('y')
plt.ylabel('y_pred')

In [ ]:
def plot_dt_exp_comparison(yexp, ypred):

    fig, axs = plt.subplots(nrows = 3, ncols = 3, figsize = [10,8], dpi = 100, sharex=True, sharey=True)
    for i in range(9):
        plt.sca(axs.flatten()[i])
        plt.plot(ypred[i], '.-', lw = 1, c = 'k', alpha = 0.5, label = 'digital twin prediction')
        plt.plot(yexp[i], '.-', lw = 1, c = 'b', alpha = 0.5, label = 'experimental outcome')
        plt.xlabel('# pendulum')
        plt.ylabel('output angle')
        plt.title(f'Initial conditions {i}')
        plt.grid()
    plt.legend()
    plt.tight_layout()
    plt.savefig('img/coupled_pendula_dt_examples.png')
    plt.show()
    
plot_dt_exp_comparison(y_test_new,5*test_out.detach())

In [ ]:

class Net2(nn.Module):

    def __init__(self):
        super(Net2, self).__init__()

        self.layer1 = nn.Linear(14, 120)
        self.layer2 = nn.Linear(120, 120)
        self.layer3 = nn.Linear(120, 120)
        self.layer4 = nn.Linear(120, 84)
        self.layer5 = nn.Linear(84, 42)
        self.layer6 = nn.Linear(42, 7)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        
        x = torch.flatten(x, 1) # flatten all dimensions except the batch dimension
        x = F.relu(self.layer1(x))
        x = x+F.relu(self.layer2(x))
        x = x+F.relu(self.layer3(x))
        x = F.relu(self.layer4(x))
        x = F.tanh(self.layer5(x))
        x = F.relu(self.layer6(x))
        return x


net2 = Net2()
print(net2)

params2 = list(net2.parameters())
print(len(params2))
print(params2[0].size())

In [ ]:
x_data=torch.tensor(x)
input = x_data
out = net2(input)
print(out)
print(out.shape)

In [ ]:
target=y1[0:1000,:,:]
target_new=target.view(1000,7)

In [ ]:
import torch.optim as optim
optimizer = torch.optim.Adam(params, lr=0.001, betas=(0.9, 0.999), eps=1e-06, weight_decay=0.1, amsgrad=False)  #adam optimizer
    
for i in range(200):
    optimizer.zero_grad()   # zero the gradient buffers
    output = net2(input)
    loss = criterion(output, target_new)
    loss.backward()
    optimizer.step()
    print(loss)
